In [60]:
import os
import sqlite3
import pandas as pd
import datetime
import numpy as np

In [62]:
##Path of the data
data_path={'menu_item':os.path.join(folder_path,'MenuItem.csv'),
           'menu_page':os.path.join(folder_path,'MenuPage.csv'),
           'menu':os.path.join(folder_path,'Menu-csv.csv'),
           'dish':os.path.join(folder_path,'Dish.csv')}

In [63]:
##Read data
menu_item=pd.read_csv(data_path['menu_item'])
menu_page=pd.read_csv(data_path['menu_page'])
menu=pd.read_csv(data_path['menu'])
dish=pd.read_csv(data_path['dish'])

In [64]:
dish.head()

,id,name,description,menus_appeared,times_appeared,first_appeared,last_appeared,lowest_price,highest_price
0,1,Consomme printaniere royal,NaN,8,8,1897,1927,0.20,0.4
1,2,Chicken gumbo,NaN,111,117,1895,1960,0.10,0.8
2,3,Tomato aux croutons,NaN,13,13,1893,1917,0.25,0.4
3,4,Onion au gratin,NaN,41,41,1900,1971,0.25,1.0
4,5,St. Emilion,NaN,66,68,1881,1981,0.00,18.0


In [65]:
menu.head()

,id,name,sponsor,event,venue,place,physical_description,occasion,notes,call_number,keywords,language,date,location,location_type,currency,currency_symbol,status,page_count,dish_count
0,12463,NaN,Hotel Eastman,breakfast,commercial,"HOT SPRINGS, AR",CARD; 4.75X7.5;,EASTER,NaN,1900-2822,NaN,NaN,1900-04-15,Hotel Eastman,NaN,NaN,NaN,complete,2,67
1,12464,NaN,Republican House,dinner,commercial,"MILWAUKEE, WI",CARD; ILLUS; COL; 7.0X9.0;,EASTER,wedgewood blue card; white embossed greek key ...,1900-2825,NaN,NaN,1900-04-15,Republican House,NaN,NaN,NaN,complete,2,34
2,12465,NaN,Norddeutscher Lloyd Bremen,breakfast,commercial,DAMPFER KAISER WILHELM DER GROSSE,CARD; ILLU; COL; 5.5X8.0;,NaN,"menu in german and english; illus, steamship a...",1900-2827,NaN,NaN,1900-04-16,Norddeutscher Lloyd Bremen,NaN,NaN,NaN,complete,2,84
3,12466,NaN,Norddeutscher Lloyd Bremen,lunch,commercial,DAMPFER KAISER WILHELM DER GROSSE,CARD; ILLU; COL; 5.5X8.0;,NaN,"menu in german and english; illus, harbor scen...",1900-2828,NaN,NaN,1900-04-16,Norddeutscher Lloyd Bremen,NaN,NaN,NaN,complete,2,63
4,12467,NaN,Norddeutscher Lloyd Bremen,dinner,commercial,DAMPFER KAISER WILHELM DER GROSSE,FOLDER; ILLU; COL; 5.5X7.5;,NaN,"menu in german and english; illus, harbor scen...",1900-2829,NaN,NaN,1900-04-16,Norddeutscher Lloyd Bremen,NaN,NaN,NaN,complete,4,33


In [66]:
menu_item.head()

,id,menu_page_id,price,high_price,dish_id,created_at,updated_at,xpos,ypos
0,1,1389,0.4,NaN,1.0,2011-03-28 15:00:44 UTC,2011-04-19 04:33:15 UTC,0.111429,0.254735
1,2,1389,0.6,NaN,2.0,2011-03-28 15:01:13 UTC,2011-04-19 15:00:54 UTC,0.438571,0.254735
2,3,1389,0.4,NaN,3.0,2011-03-28 15:01:40 UTC,2011-04-19 19:10:05 UTC,0.140000,0.261922
3,4,1389,0.5,NaN,4.0,2011-03-28 15:01:51 UTC,2011-04-19 19:07:01 UTC,0.377143,0.262720
4,5,3079,0.5,1.0,5.0,2011-03-28 15:21:26 UTC,2011-04-13 15:25:27 UTC,0.105714,0.313178


In [67]:
menu_page.head()

,id,menu_id,page_number,image_id,full_height,full_width,uuid
0,119,12460,1.0,1603595,7230.0,5428.0,510d47e4-2955-a3d9-e040-e00a18064a99
1,120,12460,2.0,1603596,5428.0,7230.0,510d47e4-2956-a3d9-e040-e00a18064a99
2,121,12460,3.0,1603597,7230.0,5428.0,510d47e4-2957-a3d9-e040-e00a18064a99
3,122,12460,4.0,1603598,7230.0,5428.0,510d47e4-2958-a3d9-e040-e00a18064a99
4,123,12461,1.0,1603591,7230.0,5428.0,510d47e4-2959-a3d9-e040-e00a18064a99


In [68]:
(menu.language.notnull()).sum()

0

In [69]:
(menu.keywords.notnull()).sum()

0

In [70]:
(menu.location_type.notnull()).sum()

0

In [71]:
##Drop columns in menu with all null values
menu=menu.drop(['language','keywords','location_type'],axis=1)

In [72]:
menu.shape

(17545, 17)

In [73]:
dish.description.notnull().sum()

0

In [74]:
##Drop columns in dish with all null values
dish=dish.drop(['description'],axis=1)

In [75]:
dish.shape

(426124, 8)

In [76]:
##Check if dates in menu are valid
date_null=menu.notnull()['date']
count=0
incorrect=[]
for n,i in enumerate(menu.date):
    if date_null[n]:
        y,m,d=str(i).split('-')
    
        valid_date=True
    
        try:
            datetime.datetime(int(y),int(m),int(d))
        except ValueError:
            valid_date=False
        
        if(valid_date):
            count+=1
        else:
            incorrect.append(n)

if not incorrect:
    print('Dates are in correct format')

Dates are in correct format


## Load pandas dataframes to SQLite database

In [129]:
connect=sqlite3.connect(sqlite_file)

In [130]:
c=connect.cursor()

In [80]:
dish_schema=list(dish)
dish_schema

['id',
 'name',
 'menus_appeared',
 'times_appeared',
 'first_appeared',
 'last_appeared',
 'lowest_price',
 'highest_price']

In [81]:
##Create dish table in the database
c.execute('CREATE TABLE dish (id INTEGER PRIMARY KEY, name TEXT, menus_appeared INTEGER,\
           times_appeared INTEGER, first_appeared INTEGER, last_appeared INTEGER, lowest_price REAL, highest_price REAL )')

In [82]:
menu_item_schema=list(menu_item)
menu_item_schema

['id',
 'menu_page_id',
 'price',
 'high_price',
 'dish_id',
 'created_at',
 'updated_at',
 'xpos',
 'ypos']

In [83]:
##Create menu_item table in the database
c.execute('CREATE TABLE menu_item (id INTEGER PRIMARY KEY, menu_page_id INTEGER, price REAL, high_price REAL, dish_id REAL,\
           created_at TEXT, updated_at TEXT, xpos REAL, ypos REAL)')

In [84]:
menu_page_schema=list(menu_page)
menu_page_schema

['id',
 'menu_id',
 'page_number',
 'image_id',
 'full_height',
 'full_width',
 'uuid']

In [85]:
##Create menu_page table in the database
c.execute('CREATE TABLE menu_page (id INTEGER PRIMARY KEY, menu_id INTEGER, page_number INTEGER, image_id INTEGER,\
           full_height REAL, full_width REAL, uuid TEXT)')

In [86]:
menu_schema=list(menu)
menu_schema

['id',
 'name',
 'sponsor',
 'event',
 'venue',
 'place',
 'physical_description',
 'occasion',
 'notes',
 'call_number',
 'date',
 'location',
 'currency',
 'currency_symbol',
 'status',
 'page_count',
 'dish_count']

In [87]:
##Create menu table in the database
c.execute('CREATE TABLE menu (id INTEGER PRIMARY KEY, name TEXT, sponsor TEXT, event TEXT, venue TEXT, place TEXT,\
          physical_description TEXT, occasion TEXT, notes TEXT, call_number TEXT, date TEXT, location TEXT,\
          currency TEXT, currency_symbol TEXT, status TEXT, page_count INTEGER, dish_count INTEGER)')

## Load data into the created databases

In [88]:
##Dish
subset=dish[list(dish)]

In [89]:
tuples = [tuple(x) for x in subset.values]

In [90]:
list(dish)

['id',
 'name',
 'menus_appeared',
 'times_appeared',
 'first_appeared',
 'last_appeared',
 'lowest_price',
 'highest_price']

In [91]:
c.executemany("insert into dish(id,name,menus_appeared,times_appeared,first_appeared,last_appeared,lowest_price,\
               highest_price) values (?, ?, ?, ?, ?, ?, ?, ?)",tuples)

In [92]:
##Menu
subset=menu[list(menu)]

In [93]:
tuples=[tuple(x) for x in subset.values]

In [94]:
list(menu)

['id',
 'name',
 'sponsor',
 'event',
 'venue',
 'place',
 'physical_description',
 'occasion',
 'notes',
 'call_number',
 'date',
 'location',
 'currency',
 'currency_symbol',
 'status',
 'page_count',
 'dish_count']

In [95]:
c.executemany("insert into menu (id, name, sponsor, event, venue, place, physical_description, occasion, notes, call_number,\
               date, location, currency, currency_symbol, status, page_count, dish_count)\
              values (?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?)", tuples)

In [96]:
##Menu_item
subset=menu_item[list(menu_item)]

In [97]:
tuples=[tuple(x) for x in subset.values]

In [100]:
list(menu_item)

['id',
 'menu_page_id',
 'price',
 'high_price',
 'dish_id',
 'created_at',
 'updated_at',
 'xpos',
 'ypos']

In [101]:
c.executemany('insert into menu_item (id, menu_page_id, price, high_price, dish_id, created_at, updated_at, xpos, ypos)\
               values(?,?,?,?,?,?,?,?,?)',tuples)

In [102]:
##menu_page
subset=menu_page[list(menu_page)]

In [103]:
tuples=[tuple(x) for x in subset.values]

In [104]:
list(menu_page)

['id',
 'menu_id',
 'page_number',
 'image_id',
 'full_height',
 'full_width',
 'uuid']

In [105]:
c.executemany('insert into menu_page (id, menu_id, page_number, image_id, full_height, full_width, uuid)\
           values(?,?,?,?,?,?,?)',tuples)

## IC check

In [106]:
##Dish
list(dish)

['id',
 'name',
 'menus_appeared',
 'times_appeared',
 'first_appeared',
 'last_appeared',
 'lowest_price',
 'highest_price']

In [107]:
##Check uniqueness of primary key
query='select id, count(id) as count_id from dish group by id having count(id) >1'
pd.read_sql(query,connect)

,id,count_id


In [108]:
##Check if primary key is not null
query='select id from dish where id is null'
pd.read_sql(query,connect)

,id


In [109]:
##Check if first appeared and last appeared are within 1851 and 2018
query='select * from dish where (first_appeared not between 1851 and 2018) or (last_appeared not between 1851 and 2018)'
pd.read_sql(query,connect)

,id,name,menus_appeared,times_appeared,first_appeared,last_appeared,lowest_price,highest_price
0,7,Radishes,3263,3347,1854,2928,0.00,25.00
1,15,Celery,4249,4694,1,2928,0.00,50.00
2,18,Sardines,1426,1484,1856,2928,0.00,50.00
3,33,Sliced Tomatoes,1195,1256,1873,2928,0.00,25.00
4,38,Apple Sauce,721,829,1,1987,0.00,20.00
5,96,Coffee,7746,8490,1,2928,0.00,30.00
6,103,Kippered Herring,231,236,1,1970,0.00,0.85
7,108,Bananas,1195,1213,1,2012,0.00,25.00
8,112,Fruit,1917,2004,1854,2928,0.00,40.00
9,155,Oatmeal,533,612,1,1988,0.00,20.00


In [110]:
##Check if first_appeared is less than last_appeared
query='select * from dish where first_appeared > last_appeared'
pd.read_sql(query,connect)

,id,name,menus_appeared,times_appeared,first_appeared,last_appeared,lowest_price,highest_price
0,164029,Clear beef broth,0,1,1900,0,0.25,0.25
1,204888,Hot roast beef with gravy,0,1,1900,0,0.25,0.25
2,250693,SURI LEBERLI - Shredded Calf's Liver Flambe in...,0,1,1945,0,NaN,NaN
3,250699,"SWISS MINCED VEAL, ROESTI",0,1,1945,0,NaN,NaN
4,301736,Cafe Glacee,0,2,1940,0,0.40,0.40
5,309629,Garlic Butter,0,1,1947,0,0.40,0.40


In [111]:
##Check if lowest_price is less than highest_price
query='select * from dish where lowest_price > highest_price'
pd.read_sql(query, connect)

,id,name,menus_appeared,times_appeared,first_appeared,last_appeared,lowest_price,highest_price


In [112]:
##menu_item
list(menu_item)

['id',
 'menu_page_id',
 'price',
 'high_price',
 'dish_id',
 'created_at',
 'updated_at',
 'xpos',
 'ypos']

In [113]:
##Check uniqueness of primary key
query='select id, count(id) as count_id from menu_item group by id having count(id)>1'
pd.read_sql(query,connect)

,id,count_id


In [114]:
##Check if primary key is null
query='select id from menu_item where id is null'
pd.read_sql(query,connect)

,id


In [115]:
##Check if dish_id in menu_item is present in dish table
query='select * from menu_item where dish_id not in (select id from dish)'
pd.read_sql(query,connect)

,id,menu_page_id,price,high_price,dish_id,created_at,updated_at,xpos,ypos
0,619133,51020,NaN,None,220797.0,2011-10-30 14:27:33 UTC,2011-10-30 14:27:33 UTC,0.605714,0.215599
1,837354,60235,0.20,None,329183.0,2012-03-08 23:28:32 UTC,2012-03-08 23:28:32 UTC,0.664286,0.173588
2,1047160,69117,0.45,None,395403.0,2012-08-14 09:52:01 UTC,2012-08-14 09:52:01 UTC,0.377333,0.469635


In [131]:
##Delete the above violations from table
c.execute('delete from menu_item where dish_id not in (select id from dish)')

In [117]:
query='select * from menu_item where dish_id not in (select id from dish)'
pd.read_sql(query,connect)

,id,menu_page_id,price,high_price,dish_id,created_at,updated_at,xpos,ypos


In [118]:
##menu_page
list(menu_page)

['id',
 'menu_id',
 'page_number',
 'image_id',
 'full_height',
 'full_width',
 'uuid']

In [119]:
##Check uniqueness of primary key
query='select id, count(id) as count_id from menu_page group by id having count(id)>1'
pd.read_sql(query,connect)

,id,count_id


In [120]:
##Check if primary key is null
query='select id from menu_page where id is null'
pd.read_sql(query,connect)

,id


In [121]:
##Check if page number is non-zero
query='select * from menu_page where page_number=0'
pd.read_sql(query,connect)

,id,menu_id,page_number,image_id,full_height,full_width,uuid


In [122]:
##Check if page number is less than total page count
query='select * from menu m, menu_page p where m.id=p.menu_id and p.page_number>m.page_count'
pd.read_sql(query,connect)

,id,name,sponsor,event,venue,place,physical_description,occasion,notes,call_number,...,status,page_count,dish_count,id,menu_id,page_number,image_id,full_height,full_width,uuid
0,21467,None,Mr. Samuel D. Coykendall,dinner to the holland society of new york on t...,soc,HOTEL KAATERSKILL,FOL; ILLUS; 7 x 11;,OTHER LITERARY EVENT BASED ON RIP VAN WINKLE,illus. of society seal and architectural relie...,1886-036,...,complete,2,35,26137,21467,3,4000000618,4628.0,3140.0,510d47db-1c4c-a3d9-e040-e00a18064a99
1,21725,None,Rivers And Harbors Committee,reception,govt,"ZOOLOGICAL GARDENS, CINCINNATI, OH",BOOKLET; ILLUS; COL;10X7;,OTHER,includes wines served with inividual courses; ...,1905-385,...,complete,10,20,27018,21725,32,4000015977,3043.0,4175.0,510d47db-71fe-a3d9-e040-e00a18064a99
2,24296,None,Ichthyophagous Club,sixth annual dinner,soc,HOTEL BUCKINGHAM,FOLDER; ILL; 5.75X9.5,"OTHER, CLUB ANNIVERSARY",elaborate illustration of numerous fish swimmi...,1885-024,...,complete,2,39,36704,24296,3,4000000537,3914.0,2365.0,510d47e4-347e-a3d9-e040-e00a18064a99
3,33935,The Grunewald,The Grunewald,None,None,None,31.5x25cm folded; 31.5x50cm open,None,None,1914-0636_wotm,...,complete,3,38,71863,33935,4,4066685,4566.0,3661.0,c6d5d0ca-dde6-54db-e040-e00a18063df6
4,34021,Waldorf Astoria,Waldorf Astoria,None,None,None,20x30.5cm folded; 20x61cm open,None,national leather and shoe finders association,1914-0724_wotm,...,complete,5,20,72076,34021,6,4066901,3520.0,4685.0,c6d5d0ca-debe-54db-e040-e00a18063df6
5,34967,Hotel Astor,Hotel Astor,None,None,None,None,None,the central dental association of northern new...,1913-0194_wotm,...,complete,12,18,74933,34967,13,5070368,5099.0,3985.0,c5efffd0-3059-0131-34a9-58d385a7bbd0


In [123]:
##menu
list(menu)

['id',
 'name',
 'sponsor',
 'event',
 'venue',
 'place',
 'physical_description',
 'occasion',
 'notes',
 'call_number',
 'date',
 'location',
 'currency',
 'currency_symbol',
 'status',
 'page_count',
 'dish_count']

In [124]:
##Check uniquesness of primary key
query='select id, count(id) as count_id from menu group by id having count(id)>1'
pd.read_sql(query,connect)

,id,count_id


In [125]:
##Check if primary key is null
query='select id from menu where id is null'
pd.read_sql(query,connect)

,id


In [126]:
#Check page_count if non-zero
query='select * from menu where page_count=0'
pd.read_sql(query,connect)

,id,name,sponsor,event,venue,place,physical_description,occasion,notes,call_number,date,location,currency,currency_symbol,status,page_count,dish_count


In [127]:
connect.commit()
connect.close()